In [1]:
%cd comp0087_assignment-main
%ls

/mnt/batch/tasks/shared/LS_root/mounts/clusters/ucabcmb1/code/Users/ucabcmb/SNLP/comp0087_assignment-main
'BERT summarisation.ipynb'*         data/                  process_data.py*
 abstractive_summarisation.ipynb*   download_hf_data.py*   test.ipynb*
 classifier.py*                     download_summary.py*
 config.py*                         optimisation.py*


In [14]:
%pip install torch
%pip install transformers
%pip install bert-extractive-summarizer

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from summarizer import Summarizer
import pandas as pd
import torch
import transformers
from pathlib import Path

In [3]:
def chunk_text(text, max_length=512):
    """
    Split a large piece of text into chunks that are no longer than max_length words,
    but do not cut off sentences.
    """
    # Split the text into sentences
    sentences = text.split('. ')

    # Initialize variables
    chunks = []
    current_chunk = ''

    # Loop through each sentence and add it to the current chunk until the
    # length of the current chunk exceeds the max_length
    for sentence in sentences:
        words = sentence.split()
        if len(current_chunk.split()) + len(words) <= max_length:
            current_chunk += sentence + '. '
        else:
            # Add the current chunk to the list of chunks
            chunks.append(current_chunk.strip())
            current_chunk = sentence + '. '

    # Add the final chunk to the list of chunks
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def chunked_extractive_summary(text, model):

  chunks = chunk_text(text)

  max_output_length = 512 / len(chunks)

  summaries = []
  for chunk in chunks:

    words_in_chunk = len(chunk.split())

    if words_in_chunk > max_output_length:
      ratio_ = max_output_length / words_in_chunk
      summaries.append(model(chunk, ratio=ratio_))
    else:
      summaries.append(chunk)

  return " ".join(summaries)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = Summarizer()

# Make folder if necessary
Path("data/echr/chunking").mkdir(parents=True, exist_ok=True)

cuda


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Loop through all text in data
for set_ in ["train","valid", "test"]:
    for name in ["non-anon"]:
    # for name in ["anon"]:
        print(f"{name} {set_} set")
        df = pd.read_pickle(f"data/echr/{name}_{set_}.pkl")
        if "summary" not in df.columns:
            df["summary"] = ""
        # Loop through summarise
        for i in range(len(df["text"])):

            text = df["text"][i]

            summary = chunked_extractive_summary(text, model)

            while True:

                if len(summary.split()) <= 512:
                    break

                # generate the condition
                summary = chunked_extractive_summary(summary, model)

            df["summary"][i] = summary

            # Save every 50 summaries in case of crashes
            if (i % 50==0) and (i>0):
                df.to_pickle(f"data/echr/chunking/{name}_{set_}.pkl")

        # Save summary
        df.to_pickle(f"data/echr/chunking/{name}_{set_}.pkl")